<a href="https://colab.research.google.com/github/madhu8888/AnomalyDetection/blob/main/Algo_evaluate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install import_ipynb
!pip install pyod
import import_ipynb
from sklearn.model_selection import train_test_split;
import numpy as np
import pandas as pd
from pyod.utils.example import visualize
from pyod.models.ocsvm import OCSVM
from pyod.models.pca import PCA
from pyod.models.gmm import GMM
from pyod.models.iforest import IForest
from sklearn.model_selection import train_test_split
import Data_generation as Data_generate

In [188]:
def split_data(df):
  X = df.loc[:, df.columns != "Label"]
  y= df["Label"]
  x_train , x_test, y_train, y_test = train_test_split(X, y,stratify=y, test_size=0.20)
  return x_train , x_test, y_train, y_test

In [189]:
def apply_anomaly_detection(x_train ,  x_test , clf):
  clf.fit(x_train.values)
  y_train_pred = clf.predict(x_train.values)
  y_test_pred = clf.predict(x_test.values) 
  return y_train_pred,y_test_pred

In [191]:
def calculate_metrics(predicted, actual):
  TP, FP, TN, FN = 0, 0, 0, 0
  for i in range(len(predicted)):
    if   (predicted[i] == 0) & (actual[i] == 0):
      TP += 1
    elif (predicted[i] == 0) & (actual[i] == 1):
      FP += 1
    elif (predicted[i] == 1) & (actual[i] == 1):
      TN += 1
    else:
      FN += 1
  accuracy  = (TP + TN) / (TP + FP + TN + FN) 
  precision = (TP) / (TP + FP) 
  recall    = (TP) / (TP + FN) 
  f1_score  = (2 * precision * recall) / (precision + recall)
  return accuracy, precision, recall, f1_score 

In [192]:
def evaluate(y_train,y_test,y_train_pred, y_test_pred):
  train_accuracy , train_precision, train_recall, train_f1_score = calculate_metrics(y_train_pred,y_train )
  print(train_accuracy , train_precision, train_recall, train_f1_score)
  test_accuracy , test_precision, test_recall, test_f1_score = calculate_metrics(y_test_pred, y_test )
  print(train_accuracy , train_precision, train_recall, train_f1_score)
  return train_accuracy , train_precision, train_recall, train_f1_score , test_accuracy , test_precision, test_recall, test_f1_score

In [193]:
def algo_metric_eval(df , clf):
  x_train , x_test, y_train, y_test = split_data(df);
  y_train_pred,y_test_pred = apply_anomaly_detection(x_train ,  x_test , clf);
  train_accuracy , train_precision, train_recall, train_f1_score = calculate_metrics(y_train_pred.tolist(),y_train.tolist() );
  test_accuracy , test_precision, test_recall, test_f1_score = calculate_metrics(y_test_pred.tolist(), y_test.tolist());
  return train_accuracy , train_precision, train_recall, train_f1_score, test_accuracy , test_precision, test_recall, test_f1_score;

# Above this

In [194]:
df = Data_generate.GenerateDataset(n_Gauss_comp=1, samples=100, cont=0.1, n_features=5 , num_cat=0 , missing_percent=0, cov_type="tied-sph", dep_per=0 , uni_low=10, uni_up=20, mu_low_range=20,mu_high_range=40)

In [195]:
clf = IForest(contamination=0.2,random_state=42);

In [ ]:
algo_metric_eval(df,clf)

In [166]:
# list of anomaly detection algorithms to be applied
random_state = np.random.RandomState(42)
outliers_fraction = 0.05;
# Define nine outlier detection tools to be compared
classifiers = {
    'Isolation Forest': IForest(contamination=outliers_fraction,random_state=random_state),
    'Principal Component Analysis (PCA)': PCA( contamination=outliers_fraction, random_state=random_state),
    'GMM': GMM(contamination=outliers_fraction),
    'One-class SVM (OCSVM)': OCSVM(contamination=outliers_fraction)
}

In [159]:
column_names = [];
d_set = ['Train', 'Test']
eval_met = ['Pre', 'Rec', 'Acc', 'F1']
for j in d_set:
  for k in eval_met:
    column_names.append(j+"_"+k)
df_return_metrics = pd.DataFrame( columns = column_names)

In [125]:
df_return_metrics

,Train_Pre,Train_Rec,Train_Acc,Train_F1,Test_Pre,Test_Rec,Test_Acc,Test_F1


In [83]:
x_train , x_test, y_train, y_test = split_data(df)

#y_train_pred,y_test_pred = apply_anomaly_detection(x_train , x_test)

In [84]:
clf = IForest(contamination=outliers_fraction,random_state=random_state);
y_train_pred,y_test_pred = apply_anomaly_detection (x_train, x_test, clf)


In [85]:
cont_factor = np.arange(0.05,0.55,0.05)


In [86]:
df_cont_factor = pd.DataFrame(cont_factor, columns =['Cont_factor'])

In [87]:
Cont_factor_train_acc = []
Cont_factor_train_precision = []
Cont_factor_train_recall = []
Cont_factor_train_f1_score = []

In [88]:
Cont_factor_test_acc = []
Cont_factor_test_precision = []
Cont_factor_test_recall = []
Cont_factor_test_f1_score = []

In [89]:
for i in cont_factor:
  clf = IForest(contamination=i,random_state=random_state);
  y_train_pred,y_test_pred = apply_anomaly_detection (x_train, x_test, clf)
  #print("y_train_pred")
  #print(y_train_pred)
  train_accuracy , train_precision, train_recall, train_f1_score = calculate_metrics(y_train_pred.tolist(),y_train.tolist() );
  test_accuracy , test_precision, test_recall, test_f1_score = calculate_metrics(y_test_pred.tolist(), y_test.tolist());

  # train data metrics
  Cont_factor_train_acc.append(train_accuracy)
  Cont_factor_train_precision.append(train_precision)
  Cont_factor_train_recall.append(train_recall)
  Cont_factor_train_f1_score.append(train_f1_score)
  # test data metrics
  #print("test accuracy")
  #print(test_accuracy , test_precision, test_recall, test_f1_score)
  Cont_factor_test_acc.append(test_accuracy)
  Cont_factor_test_precision.append(test_precision)
  Cont_factor_test_recall.append(test_recall)
  Cont_factor_test_f1_score.append(test_f1_score)



In [90]:
df_cont_factor['Train_acc'] = Cont_factor_train_acc
df_cont_factor['Train_prec'] = Cont_factor_train_precision
df_cont_factor['Train_recall'] = Cont_factor_train_recall
df_cont_factor['Train_F1'] = Cont_factor_train_f1_score


In [91]:
df_cont_factor['Test_acc'] = Cont_factor_test_acc
df_cont_factor['Test_prec'] = Cont_factor_test_precision
df_cont_factor['Test_recall'] = Cont_factor_test_recall
df_cont_factor['Test_F1'] = Cont_factor_test_f1_score

In [ ]:
algo_metric_eval(df, clf)

In [146]:
def algo_eval_2(df, clf):
    x_train , x_test, y_train, y_test = split_data(df)
    y_train_pred,y_test_pred = apply_anomaly_detection (x_train, x_test, clf)
    #df_return_metrics = get_return_dataframe()
    tempDf = pd.DataFrame(columns= column_names);
    # train metrics
    train_accuracy , train_precision, train_recall, train_f1_score = calculate_metrics(y_train_pred.tolist(),y_train.tolist() );
    print(train_accuracy , train_precision, train_recall, train_f1_score)							
    tempDf['Train_Acc']= train_accuracy;
    tempDf['Train_Pre']=train_precision;
    tempDf['Train_Rec'] = train_recall;
    tempDf['Train_F1']= train_f1_score;
    #test metrics
    test_accuracy , test_precision, test_recall, test_f1_score = calculate_metrics(y_test_pred.tolist(), y_test.tolist());
    tempDf['Test_Acc']=test_accuracy;
    tempDf['Test_Pre']=test_precision;
    tempDf['Test_Rec']=test_recall;
    tempDf['Test_F1']=test_f1_score;
    print(tempDf['Train_Acc'])
    #return train_accuracy , train_precision, train_recall, train_f1_score , test_accuracy , test_precision, test_recall, test_f1_score
    #df_return_metrics = pd.concat([df_return_metrics,tempDf])
    return tempDf

In [147]:
#def return_metrics(df, clf, df_return_metrics):
#  tempDf = algo_eval(df, clf)
#  df_return_metrics = pd.concat([df_return_metrics,tempDf])
  #print(tempDf );
  #return df_return_metrics

In [148]:
#return_metrics(df, clf,df_return_metrics )

0.6 1.0 0.5555555555555556 0.7142857142857143
Series([], Name: Train_Acc, dtype: float64)


In [127]:
#algo_eval(df, clf)

UnboundLocalError: ignored